In [ ]:
with open('dana.filtered.out', 'r') as file:
    gL = []
    for line in file:
        line = line.strip().split('\t')
        gL.append(line[0])
sing_count = 0
sing_list = []
for x in set(gL):
    if gL.count(x) == 1:
        sing_count += 1
        sing_list.append(x)
print(sing_count)

In [ ]:
import requests, json, time
singTreeList, singTreeListPruned, count, ti = [], [], 0, time.time()
noTree = 0
print('Fetching gene trees...')
for g in sing_list:
    url = 'http://rest.ensemblgenomes.org/genetree/member/id/'+ g + '?compara=metazoa;nh_format=display_label_composite;object_type=Translation;'
    headers = {'Content-Type':'text/x-nh'}
    tree = requests.get(url, headers=headers).text
    if "error" in tree:
        noTree += 1
    else:
        singTreeList.append(tree)
    count += 1
    if count % 200 == 0:
        print(count, 'done', round(time.time()-ti, 2), 'seconds')
print('Pruning...')
count, ti = 0, time.time()
for i in range(0,len(singTreeList)):
    try:
        t = PhyloTree(singTreeList[i])
    except:
        print('tree making failed')
        print(singTreeList[i])
    pList = []
    nodes = ['Dere', 'Dyak', 'Dsec', 'Dsim', 'Dmel', 'Dana', 'Dpse', 'Dper']
    for n in nodes:
        for node in t.traverse():
            if n in node.name:
                 pList.append(node.name)
    try:
        t.prune(pList)
    except:
        print('pruning failed')
        print(t)
    singTreeListPruned.append(t)
    count += 1
    if count % 200 == 0:
        print(count, 'done', round(time.time()-ti,2), 'seconds')

In [ ]:
singTreeListPrunedPseudo, noPseudo = [], 0
for tree in singTreeListPruned:
    pseudo = False
    for node in tree.traverse():
        if 'Dpse' in node.name:
            pseudo = True
    if pseudo == True:
        singTreeListPrunedPseudo.append(tree)
    else:
        noPseudo += 1
print(noPseudo)

In [ ]:
singTreeOneEach, singTreeMulti = [], []
for tree in singTreeListPrunedPseudo:
    Pseudo, Ana = 0,0
    for node in tree.traverse():
        if 'Dpse' in node.name:
            Pseudo += 1
        if 'Dana' in node.name:
            Ana += 1
    if Ana == 1 and Pseudo == 1:
        singTreeOneEach.append(tree)
    else:
        singTreeMulti.append(tree)

In [ ]:
len(singTreeMulti)

In [ ]:
len(singTreeOneEach)

In [ ]:
len(singTreeListPrunedPseudo)

In [ ]:
singTreeMulti[150].show()